In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Check if there's a suitable cohort directory for Breast_Cancer
print(f"Looking for a relevant cohort directory for {trait}...")

# Check available cohorts
available_dirs = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_dirs}")

# Breast cancer-related keywords
breast_cancer_keywords = ['breast', 'mammary', 'brca']

# Look for breast cancer-related directories
breast_cancer_related_dirs = []
for d in available_dirs:
    if any(keyword in d.lower() for keyword in breast_cancer_keywords):
        breast_cancer_related_dirs.append(d)

print(f"Breast cancer-related cohorts: {breast_cancer_related_dirs}")

if not breast_cancer_related_dirs:
    print(f"No suitable cohort found for {trait}.")
    # Mark the task as completed by recording the unavailability
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    # Exit the script early since no suitable cohort was found
    selected_cohort = None
else:
    # For breast cancer, the BRCA dataset is most relevant
    if 'TCGA_Breast_Cancer_(BRCA)' in breast_cancer_related_dirs:
        selected_cohort = 'TCGA_Breast_Cancer_(BRCA)'
    else:
        selected_cohort = breast_cancer_related_dirs[0]  # Use the first match if the preferred one isn't available

if selected_cohort:
    print(f"Selected cohort: {selected_cohort}")
    
    # Get the full path to the selected cohort directory
    cohort_dir = os.path.join(tcga_root_dir, selected_cohort)
    
    # Get the clinical and genetic data file paths
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Load the clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Basic info about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")


Looking for a relevant cohort directory for Breast_Cancer...
Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'T


Clinical data columns:
['AJCC_Stage_nature2012', 'Age_at_Initial_Pathologic_Diagnosis_nature2012', 'CN_Clusters_nature2012', 'Converted_Stage_nature2012', 'Days_to_Date_of_Last_Contact_nature2012', 'Days_to_date_of_Death_nature2012', 'ER_Status_nature2012', 'Gender_nature2012', 'HER2_Final_Status_nature2012', 'Integrated_Clusters_no_exp__nature2012', 'Integrated_Clusters_unsup_exp__nature2012', 'Integrated_Clusters_with_PAM50__nature2012', 'Metastasis_Coded_nature2012', 'Metastasis_nature2012', 'Node_Coded_nature2012', 'Node_nature2012', 'OS_Time_nature2012', 'OS_event_nature2012', 'PAM50Call_RNAseq', 'PAM50_mRNA_nature2012', 'PR_Status_nature2012', 'RPPA_Clusters_nature2012', 'SigClust_Intrinsic_mRNA_nature2012', 'SigClust_Unsupervised_mRNA_nature2012', 'Survival_Data_Form_nature2012', 'Tumor_T1_Coded_nature2012', 'Tumor_nature2012', 'Vital_Status_nature2012', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_BRCA', '_PANCAN_DNAMethyl_PANCA

### Step 2: Find Candidate Demographic Features

In [3]:
# Extract clinical data file path
tcga_cohort_dir = os.path.join(tcga_root_dir, "TCGA_Breast_Cancer_(BRCA)")
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(tcga_cohort_dir)

# Load clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Identify age-related columns
candidate_age_cols = [
    'Age_at_Initial_Pathologic_Diagnosis_nature2012',
    'age_at_initial_pathologic_diagnosis',
    'days_to_birth'  # Age can be calculated from days to birth
]

# Identify gender-related columns
candidate_gender_cols = [
    'Gender_nature2012',
    'gender'
]

# Preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

print("Age-related columns preview:")
print(age_preview)

# Preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("\nGender-related columns preview:")
print(gender_preview)


Age-related columns preview:
{'Age_at_Initial_Pathologic_Diagnosis_nature2012': [nan, nan, nan, nan, nan], 'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0], 'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0]}

Gender-related columns preview:
{'Gender_nature2012': [nan, nan, nan, nan, nan], 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Analyze age-related columns
age_col = None
# Check each age-related column
if 'age_at_initial_pathologic_diagnosis' in age_preview and not all(pd.isna(age_preview['age_at_initial_pathologic_diagnosis'])):
    age_col = 'age_at_initial_pathologic_diagnosis'
elif 'days_to_birth' in age_preview and not all(pd.isna(age_preview['days_to_birth'])):
    age_col = 'days_to_birth'
elif 'Age_at_Initial_Pathologic_Diagnosis_nature2012' in age_preview and not all(pd.isna(age_preview['Age_at_Initial_Pathologic_Diagnosis_nature2012'])):
    age_col = 'Age_at_Initial_Pathologic_Diagnosis_nature2012'

# Analyze gender-related columns
gender_col = None
# Check each gender-related column
if 'gender' in gender_preview and not all(pd.isna(gender_preview['gender'])):
    gender_col = 'gender'
elif 'Gender_nature2012' in gender_preview and not all(pd.isna(gender_preview['Gender_nature2012'])):
    gender_col = 'Gender_nature2012'

# Print the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features which will automatically create the trait variable and add age/gender if provided
# Use the correct cohort identified in Step 1
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Breast_Cancer_(BRCA)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data if not already loaded
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

linked_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Print preview of clinical features
print("Clinical features (first 5 rows):")
print(linked_clinical_df.head())

# 2. Process gene expression data
print("\nProcessing gene expression data...")
# Load genetic data from the same cohort directory
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check gene data shape
print(f"Original gene data shape: {genetic_df.shape}")

# Save a version of the gene data before normalization (as a backup)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df.to_csv(out_gene_data_file.replace('.csv', '_original.csv'))

# We need to transpose genetic data so genes are rows and samples are columns for normalization
gene_df_for_norm = genetic_df.copy()  # Keep original orientation for now

# Try to normalize gene symbols - adding debug output to understand what's happening
print("Attempting to normalize gene symbols...")
try:
    # First check if we need to transpose based on the data format
    # In TCGA data, typically genes are rows and samples are columns
    if gene_df_for_norm.shape[0] > gene_df_for_norm.shape[1]:
        # More rows than columns, likely genes are rows already
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm)
    else:
        # Need to transpose first
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm.T)
        
    print(f"Gene data shape after normalization: {normalized_gene_df.shape}")
    
    # Check if normalization returned empty DataFrame
    if normalized_gene_df.shape[0] == 0:
        print("WARNING: Gene symbol normalization returned an empty DataFrame.")
        print("Using original gene data instead of normalized data.")
        # Use original data
        normalized_gene_df = genetic_df
    
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_df = genetic_df

# Save gene data
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
# TCGA data uses the same sample IDs in both datasets
print("\nLinking clinical and genetic data...")
print(f"Clinical data shape: {linked_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_df.shape}")

# Find common samples between clinical and genetic data
# In TCGA, samples are typically columns in the gene data and index in the clinical data
common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) == 0:
    print("ERROR: No common samples found between clinical and genetic data.")
    # Try the alternative orientation
    common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.index))
    print(f"Checking alternative orientation: {len(common_samples)} common samples found.")
    
    if len(common_samples) == 0:
        # Use is_final=False mode which doesn't require df and is_biased
        validate_and_save_cohort_info(
            is_final=False,
            cohort="TCGA",
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True
        )
        print("The dataset was determined to be unusable for this trait due to no common samples. No data files were saved.")
else:
    # Filter clinical data to only include common samples
    linked_clinical_df = linked_clinical_df.loc[list(common_samples)]
    
    # Create linked data by merging
    linked_data = pd.concat([linked_clinical_df, normalized_gene_df[list(common_samples)].T], axis=1)
    
    print(f"\nLinked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first few columns):")
    display_cols = [trait, 'Age', 'Gender'] + list(linked_data.columns[3:5])
    print(linked_data[display_cols].head())
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"\nData shape after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in features
    print("\nChecking for bias in features:")
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort info
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=len(linked_data.columns) > 3,  # More than just trait/age/gender columns
        is_trait_available=trait in linked_data.columns,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Data from TCGA Breast Cancer (BRCA) cohort used for breast cancer analysis."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to: {out_data_file}")
        
        # Also save clinical data separately
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_columns = [col for col in linked_data.columns if col in [trait, 'Age', 'Gender']]
        linked_data[clinical_columns].to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print("The dataset was determined to be unusable for this trait. No data files were saved.")

Clinical features (first 5 rows):
                 Breast_Cancer   Age  Gender
sampleID                                    
TCGA-3C-AAAU-01              1  55.0     0.0
TCGA-3C-AALI-01              1  50.0     0.0
TCGA-3C-AALJ-01              1  62.0     0.0
TCGA-3C-AALK-01              1  52.0     0.0
TCGA-4H-AAAK-01              1  50.0     0.0

Processing gene expression data...


Original gene data shape: (20530, 1218)


Attempting to normalize gene symbols...


Gene data shape after normalization: (19848, 1218)


Gene data saved to: ../../output/preprocess/Breast_Cancer/gene_data/TCGA.csv

Linking clinical and genetic data...
Clinical data shape: (1247, 3)
Genetic data shape: (19848, 1218)
Number of common samples: 1218

Linked data shape: (1218, 19851)
Linked data preview (first 5 rows, first few columns):
                 Breast_Cancer   Age  Gender      A1BG  A1BG-AS1
TCGA-A2-A25B-01              1  39.0     0.0 -0.599274  0.350817
TCGA-BH-A0B8-11              0  64.0     0.0 -1.825574 -0.860783
TCGA-S3-AA11-01              1  67.0     0.0  0.593526  1.069817
TCGA-AN-A0FN-01              1  61.0     0.0 -0.471074  1.121317
TCGA-GM-A2D9-01              1  69.0     0.0  2.970626  1.559317



Data shape after handling missing values: (1218, 19851)

Checking for bias in features:
For the feature 'Breast_Cancer', the least common label is '0' with 114 occurrences. This represents 9.36% of the dataset.
The distribution of the feature 'Breast_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to: ../../output/preprocess/Breast_Cancer/TCGA.csv
Clinical data saved to: ../../output/preprocess/Breast_Cancer/clinical_data/TCGA.csv
